In [1]:
import pandas as pd
from qdrant_client.models import PointStruct,VectorParams,Distance 
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

d:\Internship\EnterpriseKnowledgeBased\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv(r"D:\Internship\EnterpriseKnowledgeBased\web scrape expirement\FAQ_Data.csv")
df

,question,answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...
...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...


In [3]:
#del df

In [4]:
#prepare vectors 
model = SentenceTransformer('all-mpnet-base-v2')
question_answers = df[["question","answer"]]
 
df['question_answer'] = texts = [f"{row['question']}  {row['answer']}" for _, row in question_answers.iterrows()]
df

,question,answer,question_answer
0,What is Offshore Banking Account?,Account that can be opened by Residents outsi...,What is Offshore Banking Account? Account th...
1,Who can open OBU Account?,Person resident outside Bangladesh including ...,Who can open OBU Account? Person resident ou...
2,Can customers open a joint OBU account?,"Yes, customers can open a joint OBU account. ...","Can customers open a joint OBU account? Yes,..."
3,What type of account can a customer open?,A customer can open a Fixed Deposit account i...,What type of account can a customer open? A ...
4,Documents required to open OBU account,Document check list for Individuals Copy of v...,Documents required to open OBU account Docum...
...,...,...,...
94,What is the SWIFT code of City Bank PLC?,Our SWIFT Code is CIBLBDDH. It is also known ...,What is the SWIFT code of City Bank PLC? Our...
95,"If I have IB account in one currency, can I se...","Yes. In this case, cross currency conversion ...","If I have IB account in one currency, can I se..."
96,Is there any tax or any duty applicable on the...,As per new Offshore Banking Act passed by Nat...,Is there any tax or any duty applicable on the...
97,How is City Bank offering such high-interest r...,Offshore banking basically is dependent on fo...,How is City Bank offering such high-interest r...


In [5]:
#processing the data 
def prepare_data(row):
    return {
        "id": int(row.name),  # Using the DataFrame's index as the ID
        "payload": {
            "question": row['question'],
            "answers": row['answer']
        },
        "vector": model.encode(row['question_answer']),  # Encode answers to vectors
    }

# Apply the function to the DataFrame
data_to_insert = df.apply(prepare_data, axis=1).tolist()
data_to_insert

    


[{'id': 0,
  'payload': {'question': 'What is Offshore Banking Account?',
   'answers': ' Account that can be opened by Residents outside Bangladesh in foreign currency. For example: Non-resident Bangladeshi nationals (NRBs), Persons of Bangladeshi origin (Bangladeshi holding foreign citizenship), Foreign nationals, Companies/Firms registered and operating abroad, Foreign institutional investors etc. can open this OBU account.'},
  'vector': array([ 1.29912365e-02, -3.99158671e-02,  6.66634738e-03, -2.65361853e-02,
         -1.48736183e-02,  2.62600593e-02,  5.47736771e-02,  1.60324275e-02,
         -2.15104092e-02,  2.21247654e-02, -3.68686654e-02, -8.21068212e-02,
          2.60305144e-02,  3.10610272e-02,  2.25664563e-02,  3.77094783e-02,
          2.96135042e-02, -3.98194231e-02, -2.46945098e-02, -1.06410356e-02,
         -2.62946915e-02,  3.93745825e-02, -1.48017919e-02, -4.69539175e-03,
          8.34767222e-02, -1.36666372e-02,  6.63131401e-02, -1.84422154e-02,
         -5.88465

In [6]:
#inserting data into qdrant collection 

# Connect to Qdrant
client = QdrantClient(host='localhost', port=6333)

# Choose your collection name
collection_name = 'city_bank_faq_data'

# Create collection if it doesn't exist
if collection_name not in client.get_collections().collections:
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            'size': len(data_to_insert[0]['vector']),  # Vector dimension
            'distance': 'Cosine'  # Distance metric for vector comparison
        }
    )

# Insert data
for data in data_to_insert:
    client.upsert(
        collection_name=collection_name,
        points=[{
            "id": data['id'],
            "vector": data['vector'],
            "payload": data['payload']
        }]
    )


In [7]:
#search query 
def search_query(query:str):
    query_vector = model.encode(query)
    search_results = client.search(
        collection_name = collection_name,
        query_vector = query_vector,
        query_filter=None,
        limit = 5
    )
    return search_results

In [8]:
query = "What is Offshore Banking Account?"
query2 = "What information is required to send remittances to any City Bank account from abroad?"
results = search_query(query2)
results

[ScoredPoint(id=36, version=36, score=0.8223108, payload={'answers': ' To send remittances, please use below information for beneficiary- SWIFT Code of City Bank\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : CIBLBDDH Bank Name\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : City Bank PLC Customer’s OBU Account Number\xa0\xa0\xa0\xa0: ………………. Account Name\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 : ……………………', 'question': 'What information is required to send remittances to any City Bank account from abroad?'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=89, version=89, score=0.81574214, payload={'answers': ' To send remittances, please use below information for beneficiary- SWIFT Code of City Bank\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\